In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from langchain_community.embeddings import OllamaEmbeddings
from langchain_qdrant import Qdrant

# --- Configuration ---
QDRANT_URL = "http://localhost:6333"  # If running locally
COLLECTION_NAME = "playground_vectors"

# --- Initialize Clients ---
qdrant_client = QdrantClient(url=QDRANT_URL)
embedder = OllamaEmbeddings(model="nomic-embed-text")



C:\Users\muhus\AppData\Local\Temp\ipykernel_25324\581208188.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embedder = OllamaEmbeddings(model="nomic-embed-text")


In [3]:
def insert_texts(texts: list, metadatas: list = None):
    vector_store = Qdrant(
        client=qdrant_client,
        collection_name=COLLECTION_NAME,
        embeddings=embedder,
    )
    vector_store.add_texts(texts=texts, metadatas=metadatas)
    print(f"✅ Inserted {len(texts)} documents into '{COLLECTION_NAME}'")


In [4]:
def create_collection():
    collections = [c.name for c in qdrant_client.get_collections().collections]
    if COLLECTION_NAME not in collections:
        qdrant_client.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(size=768, distance=Distance.COSINE)
        )
        print(f"✅ Created new collection: {COLLECTION_NAME}")
    else:
        print(f"ℹ️ Collection '{COLLECTION_NAME}' already exists.")



In [5]:
def search_similar(query: str, k: int = 3):
    vector_store = Qdrant(
        client=qdrant_client,
        collection_name=COLLECTION_NAME,
        embeddings=embedder,
    )
    results = vector_store.similarity_search(query, k=k)
    for i, doc in enumerate(results, 1):
        print(f"\n🔹 Result {i}:")
        print(f"Content: {doc.page_content}")
        print(f"Metadata: {doc.metadata}")
    return results


In [6]:
def update_document(id: int, new_text: str, new_metadata: dict = None):
    embedding = embedder.embed_query(new_text)
    qdrant_client.upsert(
        collection_name=COLLECTION_NAME,
        points=[{
            "id": id,
            "vector": embedding,
            "payload": new_metadata or {"updated": True}
        }]
    )
    print(f"🆙 Updated document with ID {id}")



In [7]:
def delete_document(id: int):
    qdrant_client.delete(collection_name=COLLECTION_NAME, points_selector={"points": [id]})
    print(f"❌ Deleted document with ID {id}")


In [8]:
def delete_collection():
    qdrant_client.delete_collection(COLLECTION_NAME)
    print(f"🗑️ Deleted collection '{COLLECTION_NAME}'")


In [10]:
# Step 1: Create collection
create_collection()

# # Step 2: Insert sample data
# insert_texts(
#     ["Qdrant is a vector database.", "Ollama provides local LLMs.", "LangChain connects everything!"],
#     [{"source": "wiki"}, {"source": "blog"}, {"source": "doc"}]
# )

# # Step 3: Search for something
# search_similar("What is Ollama?")

# # Step 4: Update a document
# update_document(1, "Ollama is a framework to run local AI models.")

# # Step 5: Delete a document
# delete_document(2)

# # Step 6: Drop the whole collection
# delete_collection()


✅ Created new collection: playground_vectors
